In [7]:
import pyspark
import os
import sys
from pyspark import SparkContext
from pyspark import SparkConf
#from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum, count

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [12]:
from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# Create a DataFrame
df = spark.createDataFrame([
(1, "Alice"),
(2, "Bob"),
(3, "Carol"),
(4, "Dave"),
(5, "Eve")
], ["id", "name"])
print(df.count())#Q2
df.show()#Q2
df.write.format("csv").mode('overwrite').save('output')#Q4 #gives df in output
group_df3 = df.groupBy("name").agg(sum("id").alias("Sums"),avg("id").alias("Avgs"))#Q3
group_df3.show()#Q3
# Filter the DataFrame to only include rows where the name starts with "A"
df = df.filter(df["name"].startswith("A"))#Q1
# Add a new column to the DataFrame called "age"
df = df.withColumn("age", df["id"] * 10)#Q1
# Print the DataFrame
df.show()


5
+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Carol|
|  4| Dave|
|  5|  Eve|
+---+-----+

+-----+----+----+
| name|Sums|Avgs|
+-----+----+----+
|Alice|   1| 1.0|
|  Bob|   2| 2.0|
|Carol|   3| 3.0|
| Dave|   4| 4.0|
|  Eve|   5| 5.0|
+-----+----+----+

+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 10|
+---+-----+---+



In [10]:
data2 = [("New York", 10.0), ("New York", 12.0),
        ("Los Angeles", 20.0), ("Los Angeles", 22.0),
        ("San Francisco", 15.0), ("San Francisco", 18.0)]
df2 = spark.createDataFrame(data2, ["city", "temperature"])
group_df2 = df2.groupBy("city").agg(sum("temperature").alias("Sums"),avg("temperature").alias("Avgs"))
df2.show()
group_df2.show()

+-------------+-----------+
|         city|temperature|
+-------------+-----------+
|     New York|       10.0|
|     New York|       12.0|
|  Los Angeles|       20.0|
|  Los Angeles|       22.0|
|San Francisco|       15.0|
|San Francisco|       18.0|
+-------------+-----------+

+-------------+----+----+
|         city|Sums|Avgs|
+-------------+----+----+
|     New York|22.0|11.0|
|  Los Angeles|42.0|21.0|
|San Francisco|33.0|16.5|
+-------------+----+----+



In [13]:
#Q5

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# Load the text file
lines = spark.read.text("sentence.txt")
# Split the lines into words
words = lines.withColumn('word', f.explode(f.split(f.col('value'), ' '))).groupBy('word').count()\
.sort('count', ascending=False)\
.show()
# Stop the SparkSession
spark.stop()

+-----+-----+
| word|count|
+-----+-----+
|   To|    2|
|   to|    2|
|seven|    1|
|three|    1|
+-----+-----+



In [19]:
#Alt
# Create a SparkSession
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
# Load the text file
lines = sc.textFile("sentence.txt")
counts = lines.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
output = counts.collect()
for (word, count) in output:
    print("%s: %i" % (word, count))
# Split the lines into words

/home/lplab/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


seven: 1
three: 1
To: 2
to: 2
